### TFRecord for Tabular Data

refernce: [Medium Article](https://medium.com/ymedialabs-innovation/how-to-use-tfrecord-with-datasets-and-iterators-in-tensorflow-with-code-samples-ffee57d298af)


In [ ]:
import tensorflow as tf

In [2]:
# make tfrec for alphanumeric data

# prepare data observations as array of dictionary of {feature:value} pairs
data_arr = [
    {
        'int_data': 108,
        'float_data': 2.45,
        'str_data': 'My String 100',
        'float_list_data': [256.78, 13.9]
    },
    {
        'int_data': 37,
        'float_data': 84.3,
        'str_data': 'My String 200',
        'float_list_data': [1.34, 843.9, 65.22]
    }
]

def get_example_object(data_record):
    ''' create example from given data record '''
    # Convert individual data into a list of int64 or float or bytes
    int_list1 = tf.train.Int64List(value = [data_record['int_data']])
    float_list1 = tf.train.FloatList(value = [data_record['float_data']])
    # Convert string data into list of bytes
    str_list1 = tf.train.BytesList(value = [data_record['str_data'].encode('utf-8')])
    float_list2 = tf.train.FloatList(value = data_record['float_list_data'])

    # Create a dictionary with above lists individually wrapped in Feature
    feature_key_value_pair = {
        'int_list1': tf.train.Feature(int64_list = int_list1),
        'float_list1': tf.train.Feature(float_list = float_list1),
        'str_list1': tf.train.Feature(bytes_list = str_list1),
        'float_list2': tf.train.Feature(float_list = float_list2)
    }

    # Create Features object with above feature dictionary
    features = tf.train.Features(feature = feature_key_value_pair)

    # Create Example object with features
    example = tf.train.Example(features = features)
    return example

with tf.io.TFRecordWriter('example.tfrecord') as tfwriter:
    # Iterate through all data
    for data_record in data_arr:
        # make example object
        example = get_example_object(data_record)
        # Append each example into tfrecord
        tfwriter.write(example.SerializeToString())

In [3]:
def extract_fn(data_record):
    features = {
        # Extract features using the keys set during creation
        'int_list1': tf.io.FixedLenFeature([], tf.int64),
        'float_list1': tf.io.FixedLenFeature([], tf.float32),
        'str_list1': tf.io.FixedLenFeature([], tf.string),
        # If size is different of different records, use VarLenFeature 
        'float_list2': tf.io.VarLenFeature(tf.float32)
    }
    sample = tf.io.parse_single_example(data_record, features)
    return sample

# Initialize all tfrecord paths
dataset = tf.data.TFRecordDataset(['example.tfrecord']) # provide list of tfrecord paths
dataset = dataset.map(extract_fn) # map extracting func to dataset
iterator = iter(dataset) # create iterator for elements in dataset

# next_element = iterator.get_next()
for item in iterator:
  print(item['int_list1'].numpy())
  print(item['float_list1'].numpy())
  print(item['str_list1'].numpy().decode())
  print(item['float_list2'].values.numpy())
  print()

2023-05-18 20:32:16.529993: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-18 20:32:16.530485: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-18 20:32:16.530516: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (zephyr): /proc/driver/nvidia/version does not exist


108
2.45
My String 100
[256.78  13.9 ]

37
84.3
My String 200
[  1.34 843.9   65.22]

